<a href="https://colab.research.google.com/github/kaspergroenbek98/first-first-year-project/blob/master/Copy_of_SklearnNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/kaspergroenbek98/first-first-year-project.git


fatal: destination path 'first-first-year-project' already exists and is not an empty directory.


In [2]:
import numpy as np
import nltk
import csv
import re
import pandas as pd
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
# Using pandas to load the data file into a CSV.
# ISO-8859-1 is the encoding for tweets.
df = pd.read_csv("first-first-year-project/data.csv", encoding = "ISO-8859-1")
#First step to cleaning the data, removing the categories we dont need.
first_clean = df.drop(["other_topic","resolution_topics", "tweet_coord", "tweet_created","tweet_id", "name", "retweet_count", "tweet_date", "user_timezone"], axis = 1)
first_clean


,gender,Resolution_Category,text,tweet_location,tweet_state,tweet_region
0,female,Health & Fitness,"#NewYearsResolution :: Read more books, No scr...",Southern California,CA,West
1,female,Humor,#NewYearsResolution Finally master @ZJ10 's pa...,New Jersey,NJ,Northeast
2,male,Personal Growth,#NewYearsResolution to stop being so damn perf...,Hollywood,CA,West
3,male,Philanthropic,My #NewYearsResolution is to help my disabled ...,Metro NYC,NY,Northeast
4,female,Personal Growth,#NewYearsResolution #2015Goals #2015bucketlist...,"Pittsburgh, Pennsylvania",PA,Northeast
...,...,...,...,...,...,...
5006,female,Recreation & Leisure,Tomorrow I start @JustifiedFX because @natalie...,NC/TN,TN,South
5007,female,Humor,"holy crap, people. EYES OPEN WHEN DRIVING. #Ne...","charleston, nyc",NY,Northeast
5008,female,Humor,RT @moJO_SHabby: Start parody of her blog #NYR...,Memphis,TN,South
5009,female,Career,RT @kscmaghirang: To have an excellent job bef...,Paris USA,TX,South


In [0]:
#Time for RegEx cleaning/processing.
def cleaner(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub('b4', 'before', text)
    text = re.sub(r'@[A-Za-z0-9]+','',text)
    text = re.sub('https?://[A-Za-z0-9./]+','', text)
    text = text.strip(' ')
    return text

In [29]:
#Cleaning all the text in our data with our newly build cleaner function.
for i in range(len(first_clean.text)):
  first_clean.text[i] = cleaner(first_clean.text[i])

from sklearn.model_selection import train_test_split
#Splitting the data into train and test set. + shuffling while doing so. the test size is = 20%.
train, test = train_test_split(first_clean, random_state=42, test_size=0.2, shuffle=True)




1973    ûï chrisbrown supper club right now newyearsre...
3666    rt goltv bae i want a man with a career i am o...
831     newyearsresolution stop getting hit in the fac...
239     newyearsresolution continue to date hilariousl...
4692                    find more hoes newyearsresolution
                              ...                        
4426    i have newyearsresolution ideas amp really sho...
466        newyearsresolution 1 give up chocolate for jan
3092    new years resolution not to have a new years r...
3772    i do not usually indulge in them but i have go...
860               newyearsresolution stop burning barbies
Name: text, Length: 4008, dtype: object

1052    ûï cinemax newyearsresolution be more badass h...
3074    stuck with my new years resolution for 2 days ...
1789    kalanisim55 be a badass as usual newyearsresol...
764     rt rockii96 newyearsresolution go to the gym o...
3822    feeling fat thinking about doing sprints amp e...
                              .

In [0]:
#Create a list of ALL tweets. (Each element in the list is a tweet)
sequence = list()
for i in range(len(cleaned_tweets)):
  sequence.append(cleaned_tweets[i].strip())

#Using Sklearn, playing around with the CountVectorizer.
vectorizer = CountVectorizer(analyzer="word", lowercase=True, stop_words= "english",min_df= 5, binary= True)
X = vectorizer.fit_transform(sequence)
X.toarray()
print(vectorizer.get_feature_names())
print(vectorizer.vocabulary_.get(u"newyearsresolution"))
print(X.toarray())
print(X.shape)

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
#Here we use term frequency to downscale the importance of words occuring many times in a tweet. That way we hopefully get more weighted words. 
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X)
X_train_tfidf.shape


In [0]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, first_clean.Resolution_Category )
